In [16]:
import sys
sys.path.append('../src/')

import data_io
import pandas as pd
import numpy as np
import re

In [17]:
unique_locations = pd.read_csv(data_io.input_cleaned/'geolocations'/
                            'unique_locations_w_fips_scraped.csv',
                              encoding='utf-8',
                              dtype={'state_county_fips_str':'str'})
unique_locations.dropna(subset=["state_county_fips_str"],inplace=True)
print(len(unique_locations))
unique_locations.head()

11823


,campaign_id,location_city,location_stateprefix,location_statefullname,location_city_only,location_tosearch,state_county_fips_str,county_name,search_status,location_lat,location_lng
0,5cf2pqo,"Dayton, OH",OH,Ohio,Dayton,"Dayton, Ohio",39113,Montgomery,NaN,39.7589,-84.1919
1,6wxxj-steve-mcallister,"Lilburn, GA",GA,Georgia,Lilburn,"Lilburn, Georgia",13135,Gwinnett,NaN,33.8901,-84.1430
2,HelptheShoeDoctor,"San Leandro, CA",CA,California,San Leandro,"San Leandro, California",06001,Alameda,NaN,37.7249,-122.1561
3,HopeforlifeSharith,"Revere, MA",MA,Massachusetts,Revere,"Revere, Massachusetts",25025,Suffolk,NaN,42.4084,-71.0120
4,LeslieCarey,"Wilmington, DE",DE,Delaware,Wilmington,"Wilmington, Delaware",10003,New Castle,NaN,39.7459,-75.5466


In [18]:
df = pd.read_csv(data_io.input_cleaned/'gfm'/'US_cancer_campaigns_2018_2021.csv',index_col=[0],
                 sep='|',encoding='utf-8')
df.head()

,url,last_donation_time,last_update_time,location_city,location_country,location_stateprefix,poster,story,title,raised_amnt,...,tag,num_donors,num_likes,num_shares,day,month,year,location_city_only,cancer_in_story,cancer_in_title
campaign_id,,,,,,,,,,,,,,,,,,,,,
5cf2pqo,https://www.gofundme.com/f/5cf2pqo,33 mos,2018-07-16T01:05:46-05:00,"Dayton, OH",US,OH,Valerie Roof,"As many of you already know, Jess recently had...","Help Jimmy Walker, R.I.P. Jess",3500.0,...,"Medical, Illness & Healing",53.0,NaN,NaN,1.0,April,2018.0,Dayton,True,False
6wxxj-steve-mcallister,https://www.gofundme.com/f/6wxxj-steve-mcallister,33 mos,2018-10-27T15:04:49-05:00,"Lilburn, GA",US,GA,Betty Kinnett,I have a heavy heart today and I need your pra...,Steve McAllister,11009.0,...,"Medical, Illness & Healing",98.0,NaN,NaN,1.0,April,2018.0,Lilburn,True,False
HelptheShoeDoctor,https://www.gofundme.com/f/HelptheShoeDoctor,28 mos,2020-05-29T23:53:53-05:00,"San Leandro, CA",US,CA,Terry O'Neal-Feaster,Allen Feaster (Center) Fought to Close Youth P...,Help Community Activist Beat Cancer,7607.0,...,"Medical, Illness & Healing",123.0,NaN,NaN,1.0,April,2018.0,San Leandro,True,True
HopeforlifeSharith,https://www.gofundme.com/f/HopeforlifeSharith,34 mos,2019-03-28T11:16:35-05:00,"Revere, MA",US,MA,Felipe Rivas,The girl you see in the picture is my cousin S...,Help Sharith beat Brain Cancer,2024.0,...,"Medical, Illness & Healing",53.0,NaN,NaN,1.0,April,2018.0,Revere,True,True
LeslieCarey,https://www.gofundme.com/f/LeslieCarey,31 mos,2021-01-28T19:04:16-06:00,"Wilmington, DE",US,DE,Susan Oates,"Monday, April 2, 2018 ~ After bravely battling...",Help While Healing for Leslie Carey,14825.0,...,"Medical, Illness & Healing",182.0,NaN,NaN,1.0,April,2018.0,Wilmington,True,False


In [15]:
# exclusion_df = pd.read_csv(data_io.input_cleaned/'gfm'/'exclusion_tracker_rd_2.csv',
#                               index_col = 0)

In [19]:
#drop locations that didn't geocode
unique_locations = unique_locations.replace('nan',np.nan).replace('none',np.nan)
unique_locations.dropna(subset=['county_name'], inplace=True)

county_dict = dict(
    zip(unique_locations['location_city'].to_list(), unique_locations['county_name'].to_list()))
long_fips_dict = dict(zip(unique_locations['location_city'].to_list(), 
                          unique_locations['state_county_fips_str'].to_list()))


cleaned_location_city = df['location_city']
df['location_county'] = cleaned_location_city.map(county_dict)
df['location_state_county_fip'] = cleaned_location_city.map(long_fips_dict)

In [21]:
unique_locs_to_scrape = df.loc[df.location_state_county_fip.isna(),
                               ['location_city','location_city_only','location_stateprefix']].drop_duplicates()
print(unique_locs_to_scrape.shape)
unique_locs_to_scrape.to_csv(data_io.input_cleaned/'geolocations'/'unique_locations_to_scrape_again.csv')

(217, 3)


In [22]:
#exclusion_df.loc['deleted', 'failed_geocode'] = df['county'].isnull().sum()
df = df.dropna(subset = ['location_county'])
# exclusion_df.loc['total', 'failed_geocode'] = len(df)
# exclusion_df.to_csv(data_io.input_cleaned/'gfm'/'final_exclusion_tracker.csv')
geo_fail = df[pd.isnull(df['location_county'])]
#save failed geocodes to make sure nothing in the US failed

save = False
if save:
    geo_fail.to_csv(data_io.input_cleaned/'gfm'/'master_failed_geocode.csv', encoding='utf-8-sig')
df.dropna(subset=['location_county'], inplace=True)

#### Define text mining functions

In [23]:
#functions for additional text mining
SEARCH_OPTIONS = pd.read_csv(data_io.gfm/'gfm'/'free_text_search_terms_w_covid.csv')

SEARCH_DICT = {k:SEARCH_OPTIONS[k].dropna().to_list() for k in SEARCH_OPTIONS.columns}

    
def create_dict(search_type):
    key_col = 'collapsed_'+search_type
    new = SEARCH_OPTIONS.dropna(subset=[search_type])
    this_dict = pd.Series(new[key_col].values,index=new[search_type].values).to_dict()
    
    return this_dict

INSURE_DICT = create_dict('insurance_type')
#OOP_DICT = create_dict('oop_type')
import string   
#import regex as re
def extract_search_term_regex(x, search_type = 'cancer_type', return_context = False,
                             find_uninsured = False, collapse_dict = 'none'):
    if type(x) == str:
        x = x.lower()
    else:
        return np.nan
    
    search_terms = SEARCH_DICT[search_type]
    #match only if char after match is a space or punctuation
    if 'cancer' in search_type:
        for s in search_terms:
            smatch = re.search(s+'\W', x)
            if smatch:
                if return_context == True:
                    end_smatch = smatch.span()[1]
                    new = x[smatch.span()[0]:]
                    new = new[0: new.find('.')]
                    return new

                return(x[smatch.span()[0]:smatch.span()[1]])
        return np.nan
    else:
        return_val = False
        uninsure = False
        mention = []
        collapsed_mention = []
        for s in search_terms:
            smatch = re.search(s, x)
            
            if smatch:
                if return_context == True:
                    
                    new = x[smatch.span()[0]:smatch.span()[1]]
                    #print(new)
                    mention.append(new)
                
                if find_uninsured == True:
                    if INSURE_DICT[s] == 'uninsured' or INSURE_DICT[s] == 'underunisured':
                        return True
                else:
                    return_val = True
                
                if type(collapse_dict) != str:
                    collapsed_mention.append(collapse_dict[s])

                return_val = True
                
        if len(mention) >= 1:
            mention = ','.join(mention)
        else:
            mention = None
            
        if len(collapsed_mention) >= 1:
            collapsed_mention = np.unique(np.asarray(collapsed_mention))
            collapsed_mention = list(collapsed_mention)
            collapsed_mention = ','.join
        else:
            collapsed_mention = None
            
        if type(collapse_dict)!= str:
            return collapsed_mention
        
        return mention if return_context == True else return_val

def search_story_and_title(story, title, search_type):
    story_truth = extract_search_term_regex(story, search_type = search_type)
    title_truth = extract_search_term_regex(title, search_type = search_type)
    if title_truth == True or story_truth == True:
        return True
    elif title_truth == False and story_truth == False:
        return False

def get_all_mentions(story, title, search_type):
    story_truth = extract_search_term_regex(story, 
                                            search_type = search_type, 
                                            return_context = True)
    title_truth = extract_search_term_regex(title, 
                                            search_type = search_type, 
                                            return_context = True)
    if type(story_truth) == str:
        if type(title_truth) == str:
            story_truth += title_truth
        return story_truth
    elif type(title_truth) == str:
        return title_truth
    else:
        return None

#Returns a comma separated string of the features that match the search in question
def extract_feature(story, feature = 'tx_type_search', title = None):
    features = SEARCH_DICT[feature]
    if pd.isnull(title):
        searches = [story]
    else:
        searches = [story, title]
        
    return_str = ''
    for x in searches:
        if type(x) == str:
            x = x.lower()
            for f in features:
                if f in x:
                    if len(return_str) == 0:
                        return_str += f
                    else:
                        return_str += ', '
                        return_str += f
                        
    if len(return_str) == 0:
        return np.nan
    else:
        return return_str


def collapse_feature(mentions, feature_dict):
    if type(mentions) == str:
        temp_mentions = mentions.split(', ')
        new_mentions = []
        for t in temp_mentions:
            new_mentions.append(feature_dict[t])
        
        new_mentions = np.unique(new_mentions)
        
        new_mentions = ', '.join(new_mentions)
        return new_mentions


    
def assign_num_occurrences(mentions):
    if type(mentions) == str:
        if ',' in mentions:
            new = mentions.split(',')
            return len(new)
        else:
            if mentions != '':
                return 1
        
    else:
        return 0

####  Mine each text feature

In [34]:
#Look for clinical/financial details
recode_feats_to_search =  ['oop_type', 'insurance_type', 'tx_type', 'cancer_type']
df['story_and_title'] = df['title'] + ' ' + df['story']
for r in recode_feats_to_search:
    new_col = r + '_is_mentioned'
    print(f"searching for {r}")
    df[new_col] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                                        search_type = r),
                                   axis = 1)
    print(f"extracting {r}")
    df[r] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                                        search_type = r,
                                                        return_context = True),
                                   axis = 1)
    recode = 'collapsed_' + r

    feat_dict = create_dict(r)
    print(f"collapsing {r}")
    df[recode] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                                        search_type = r,
                                                        collapse_dict = feat_dict),
                                   axis = 1)


df['num_tx'] = df['collapsed_tx_type'].apply(assign_num_occurrences)
df['num_oop'] = df['collapsed_oop_type'].apply(assign_num_occurrences)
df['uninsured'] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                                            search_type = 'insurance_type',
                                                            find_uninsured = True),
                                   axis = 1)

#Look for mention of Covid
df["covid_is_mentioned"] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                        search_type = "covid"),
                   axis = 1)

#Look for worth indicators
worth_indicators = ['brave', 'nice', 'thank', 'self_reliance', 'battle']

for w in worth_indicators:
    new_col = w + '_is_mentioned'
    print(f"searching for {w}")
    df[new_col] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                                        search_type = w),
                                   axis = 1)
    print(f"extracting {w}")
    df[w] = df.apply(lambda x: extract_search_term_regex(x['story_and_title'],
                                                        search_type = w,
                                                        return_context = True),
                                   axis = 1)
save = True
if save:
    df.drop(columns=['story_and_title'],inplace=True)
    df.to_csv(data_io.input_cleaned/'gfm'/'US_cancer_campaigns_2018_2021_with_fips_and_text_features.csv',
              encoding='utf-8',sep='|')

searching for oop_type
extracting oop_type
collapsing oop_type
searching for insurance_type
extracting insurance_type
collapsing insurance_type
searching for tx_type
extracting tx_type
collapsing tx_type
searching for cancer_type
extracting cancer_type
collapsing cancer_type
searching for brave
extracting brave
searching for nice
extracting nice
searching for thank
extracting thank
searching for self_reliance
extracting self_reliance
searching for battle
extracting battle


In [35]:
df.columns

Index(['url', 'last_donation_time', 'last_update_time', 'location_city',
       'location_country', 'location_stateprefix', 'poster', 'story', 'title',
       'raised_amnt', 'goal_amnt', 'currency', 'tag', 'num_donors',
       'num_likes', 'num_shares', 'day', 'month', 'year', 'location_city_only',
       'cancer_in_story', 'cancer_in_title', 'location_county',
       'location_state_county_fip', 'location_county_fip',
       'oop_type_is_mentioned', 'oop_type', 'collapsed_oop_type',
       'insurance_type_is_mentioned', 'insurance_type',
       'collapsed_insurance_type', 'tx_type_is_mentioned', 'tx_type',
       'collapsed_tx_type', 'cancer_type_is_mentioned', 'cancer_type',
       'collapsed_cancer_type', 'num_tx', 'num_oop', 'uninsured',
       'brave_is_mentioned', 'brave', 'nice_is_mentioned', 'nice',
       'thank_is_mentioned', 'thank', 'self_reliance_is_mentioned',
       'self_reliance', 'battle_is_mentioned', 'battle',
       'collapsed_cancer_type_is_mentioned', 'covid'],
 

Number of campaigns that mentioned COVID vs all campaigns

In [37]:
df.covid.sum(),df.shape

(8729, (85531, 52))